# Cargar **librerias**

**Cargar datos aumentados**

In [2]:
import zipfile
import os

In [3]:

!wget --no-check-certificate \
    "https://github.com/arbeyvillegas/IntrDeepLearning/raw/main/Entrega-02/data/brain_tumor_data.zip" \
    -O "/tmp/brain_tumor_data.zip"

zip_ref = zipfile.ZipFile('/tmp/brain_tumor_data.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

--2023-11-20 01:39:37--  https://github.com/arbeyvillegas/IntrDeepLearning/raw/main/Entrega-02/data/brain_tumor_data.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/arbeyvillegas/IntrDeepLearning/main/Entrega-02/data/brain_tumor_data.zip [following]
--2023-11-20 01:39:37--  https://raw.githubusercontent.com/arbeyvillegas/IntrDeepLearning/main/Entrega-02/data/brain_tumor_data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 91021706 (87M) [application/zip]
Saving to: ‘/tmp/brain_tumor_data.zip’

/tmp/brain_tumor_da 100%[===================>]  86.80M   158MB/s    in 0.5s    

2023-11-20 01:

In [9]:
carpeta_training = "/tmp/brain_tumor_data/Training"
carpeta_testing = "/tmp/brain_tumor_data/Testing"

# **Mas librerias**

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import time
import numpy as np
import keras.optimizers
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator

In [11]:
batch_size = 32
nb_classes = 4
nb_epochs = 10
img_channel = 3
nb_filters = 32
nb_pool = 2
nb_conv = 3
img_size = 224

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, horizontal_flip=True,)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(carpeta_training, target_size=(img_size, img_size), batch_size=batch_size, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(carpeta_testing, target_size=(img_size, img_size), batch_size=batch_size, class_mode='categorical')

Found 2870 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


In [12]:
steps_per_epoch = 2870//batch_size

validation_steps = 394//batch_size

# **Entrenar una red neuronal convolusional**

In [13]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [ ]:
#dense_layers = [0,1, 2]
#layer_sizes = [32, 64, 128]
#conv_layers = [1, 2, 3]
dense_layers = [1]
layer_sizes = [128]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)
            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=(img_size,img_size,img_channel)))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())
            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))
                model.add(Dropout(0.33))

            model.add(Dense(4))
            model.add(Activation('softmax'))


            model.compile(loss='categorical_crossentropy',
              optimizer= "adam",
              metrics=['accuracy'],
              )

            #Fit the model
            #model.fit(X_train, Y_train,
            #batch_size=32,
            #epochs=5,
            #validation_data=(X_test,Y_test),
            #callbacks=[tensorboard,es])
            model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=nb_epochs, validation_data=validation_generator, validation_steps=validation_steps, callbacks=[tensorboard,es])



3-conv-128-nodes-1-dense-1700445069
Epoch 1/10
89/89 [==============================] - 1256s 14s/step - loss: 1.1106 - accuracy: 0.5243 - val_loss: 1.7392 - val_accuracy: 0.3490
Epoch 2/10
89/89 [==============================] - 1212s 14s/step - loss: 0.8395 - accuracy: 0.6399 - val_loss: 1.9149 - val_accuracy: 0.4062
Epoch 3/10
89/89 [==============================] - 1237s 14s/step - loss: 0.6978 - accuracy: 0.7061 - val_loss: 1.9444 - val_accuracy: 0.4062
Epoch 4/10
89/89 [==============================] - 1236s 14s/step - loss: 0.6116 - accuracy: 0.7371 - val_loss: 2.1222 - val_accuracy: 0.3958
Epoch 5/10
89/89 [==============================] - 1257s 14s/step - loss: 0.5845 - accuracy: 0.7611 - val_loss: 2.1626 - val_accuracy: 0.4661
Epoch 6/10
29/89 [========>.....................] - ETA: 13:09 - loss: 0.4875 - accuracy: 0.7941

In [ ]:
#Score
scores = model.evaluate(X_test, Y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

#Save model
model.save("{}-model-{}-accuracy.keras".format(NAME,scores[1]))

#precision    recall  f1-score   support
y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(Y_test, y_pred_bool))
print()
print()